## Data

In [1]:
import torch
import torch.nn as nn
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

corpus = [
    "ăn quả nhớ kẻ trồng cây",
    "có chí thì nên"    
]
data_size = len(corpus)

# Define the max vocabulary size and sequence length
vocab_size = 15
sequence_length = 7

In [2]:
# Initialize the tokenizer and define a trainer
tokenizer = Tokenizer(WordLevel())
tokenizer.pre_tokenizer = Whitespace()
tokenizer.enable_padding(pad_id=1, 
                         pad_token="<pad>", 
                         length=sequence_length)
tokenizer.enable_truncation(max_length=sequence_length)

# Train the tokenizer on your corpus
trainer = WordLevelTrainer(vocab_size=vocab_size, 
                           special_tokens=["<unk>", "<pad>", "<sos_topic1>", "<sos_topic2>", "<eos>"])
tokenizer.train_from_iterator(corpus, trainer)

In [3]:
data_x = []
data_y = []

corpus[0] = '<sos_topic1> ' + corpus[0] + ' <eos>'
corpus[1] = '<sos_topic2> ' + corpus[1] + ' <eos>'

for vector in corpus:
    vector = vector.split()
    data_x.append( ' '.join(vector[:-1]) )
    data_y.append( ' '.join(vector[1:]) )

print(data_x)
print(data_y)

['<sos_topic1> ăn quả nhớ kẻ trồng cây', '<sos_topic2> có chí thì nên']
['ăn quả nhớ kẻ trồng cây <eos>', 'có chí thì nên <eos>']


In [4]:
# Tokenize and numericalize your samples
def vectorize(x, y, tokenizer, sequence_length):     
    x_ids = tokenizer.encode(x).ids
    y_ids = tokenizer.encode(y).ids
    print(x_ids, y_ids)
    return x_ids, y_ids

# Vectorize the samples
data_x_ids = []
data_y_ids = []
for x, y in zip(data_x, data_y):
    x_ids, y_ids = vectorize(x, y, tokenizer, sequence_length)
    data_x_ids.append(x_ids)
    data_y_ids.append(y_ids)

data_x_ids = torch.tensor(data_x_ids, dtype=torch.long)
data_y_ids = torch.tensor(data_y_ids, dtype=torch.long)

[2, 14, 11, 9, 8, 13, 6] [14, 11, 9, 8, 13, 6, 4]
[3, 7, 5, 12, 10, 1, 1] [7, 5, 12, 10, 4, 1, 1]


## Train with full data

In [5]:
class TG_Model(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim)
        self.recurrent = nn.RNN(embed_dim, embed_dim, batch_first=True)
        self.linear = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x,_ = self.recurrent(x)        
        x = self.linear(x)
        return x.permute(0,2,1)

model = TG_Model(vocab_size, 8)
print(model)

TG_Model(
  (embedding): Embedding(15, 8)
  (recurrent): RNN(8, 8, batch_first=True)
  (linear): Linear(in_features=8, out_features=15, bias=True)
)


In [6]:
outputs = model(data_x_ids)
print(outputs.shape)

torch.Size([2, 15, 7])


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [8]:
# train
for _ in range(40):
    optimizer.zero_grad()
    outputs = model(data_x_ids)
    loss = criterion(outputs, data_y_ids)
    #print(loss.item())
    loss.backward()
    optimizer.step()

In [9]:
outputs = model(data_x_ids)
#print(outputs)
print(torch.argmax(outputs, axis=1))

tensor([[14, 11,  9,  8, 13,  6,  4],
        [ 7,  5, 12, 10,  4,  1,  1]])


In [10]:
data_y_ids

tensor([[14, 11,  9,  8, 13,  6,  4],
        [ 7,  5, 12, 10,  4,  1,  1]])

## Inference

In [11]:
promt = '<sos_topic1>'
promt_length = 1
promt_ids = tokenizer.encode(promt).ids
print(promt_ids)

[2, 1, 1, 1, 1, 1, 1]


In [12]:
for i in range(sequence_length - promt_length):
    promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
    outputs = model(promt_tensor)
    outputs = torch.argmax(outputs, axis=1)   
    next_id = outputs[0][promt_length+i-1]

    promt_ids[promt_length+i] = next_id.item()
    print(promt_ids)

[2, 14, 1, 1, 1, 1, 1]
[2, 14, 11, 1, 1, 1, 1]
[2, 14, 11, 9, 1, 1, 1]
[2, 14, 11, 9, 8, 1, 1]
[2, 14, 11, 9, 8, 13, 1]
[2, 14, 11, 9, 8, 13, 6]


In [13]:
print(tokenizer.get_vocab())

{'<eos>': 4, '<unk>': 0, 'kẻ': 8, 'chí': 5, 'nhớ': 9, 'thì': 12, 'cây': 6, 'trồng': 13, 'quả': 11, '<sos_topic1>': 2, '<sos_topic2>': 3, '<pad>': 1, 'có': 7, 'nên': 10, 'ăn': 14}
